In [1]:
import ensembl_rest as er
import numpy as np
import pandas as pd
from itertools import chain, combinations
from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)

In [6]:
ag = pd.read_csv('data/raw/ensembl_interim.csv')
au = pd.read_csv('data/raw/ensembl_uniprot.txt')
ap = pd.read_csv('data/raw/ensembl_ucsc.txt')

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14780\3056214052.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ag = pd.read_csv('data/raw/ensembl_interim.csv')


In [25]:
ag = ag.rename({'Transcript stable ID': 'ensembl_trs_id',
                'Gene stable ID': 'ensembl_gene_id',
                'Gene name': 'ensembl_gene_name',
                'Protein stable ID': 'ensembl_protein_id',
                'Chromosome/scaffold name': 'chromosome_or_scaffold',
                'Gene start (bp)': 'gene_start',
                'Gene end (bp)': 'gene_end',
                'Strand': 'strand',
                'Transcript start (bp)': 'trs_start',
                'Transcript end (bp)': 'trs_end',
                'Transcript length (including UTRs and CDS)': 'trs_length_bp',
                'Transcript support level (TSL)': 'trs_support_level',
                'GENCODE basic annotation': 'in_gencode_basic',
                'APPRIS annotation': 'appris_annotation',
                'Ensembl Canonical': 'is_ensembl_canonical',
                'Transcript name': 'trs_name',
                'Transcript count': 'trs_count',
                'Gene type': 'gene_type',
                'Transcript type': 'trs_type'}, axis=1)

In [29]:
ag = ag.rename({'trs_name': 'ensembl_trs_name'}, axis=1)

In [27]:
ag = ag.drop(['Transcription start site (TSS)', 'trs_count'], axis=1)

In [30]:
ag = ag[['chromosome_or_scaffold', 'ensembl_gene_id', 'ensembl_gene_name', 'gene_type', 'strand', 'gene_start', 'gene_end', 'ensembl_trs_id', 'ensembl_trs_name', 'trs_type', 'is_ensembl_canonical', 'trs_support_level', 'trs_length_bp', 'trs_start', 'trs_end', 'in_gencode_basic', 'appris_annotation']]

In [33]:
ag['is_ensembl_canonical'] = ag['is_ensembl_canonical'].map(lambda x: True if x == 1.0 else False)

In [35]:
ag['in_gencode_basic'] = ag['in_gencode_basic'].map(lambda x: True if x == 'GENCODE basic' else False)

In [40]:
ag['trs_support_level'] = ag['trs_support_level'].fillna('tslNA')

In [42]:
ag['trs_support_level'] = ag['trs_support_level'].map(lambda x: x[:x.index(' ')] if ' ' in x else x)

In [46]:
ag['appris_annotation'].unique()

array([nan, 'principal1', 'alternative2', 'principal2', 'principal4',
       'alternative1', 'principal3', 'principal5'], dtype=object)

In [4]:
ag = ag.drop('Unnamed: 0', axis=1)

In [5]:
ag.to_csv('data/raw/ensembl_interim.csv', index=False)

In [13]:
au['uniprot_trembl_id'] = au.groupby('Transcript stable ID')['UniProtKB/TrEMBL ID'].transform(lambda x: ', '.join(sorted([i for i in x if i is not np.nan])))

In [15]:
au['UniProtKB/Swiss-Prot ID'] = au['UniProtKB/Swiss-Prot ID'].map(lambda x: x + '-1' if x is not np.nan else np.nan)

In [16]:
au['UniProtKB isoform ID'] = au['UniProtKB isoform ID'].combine_first(au['UniProtKB/Swiss-Prot ID'])

In [28]:
len(au2.drop_duplicates('UniProtKB isoform ID')) - len(au2.drop_duplicates('Transcript stable ID'))

-14612

In [45]:
au2 = au.dropna(subset='UniProtKB isoform ID')

In [46]:
au2 = au2.drop(['UniProtKB/TrEMBL ID'], axis=1)

In [47]:
au2

,Gene stable ID,Transcript stable ID,Protein stable ID,UniProtKB isoform ID,UniProtKB/Swiss-Prot ID,uniprot_trembl_id
0,ENSG00000198888,ENST00000361390,ENSP00000354687,P03886-1,P03886-1,U5Z754
1,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,"A0A1X7RBG6, Q7GXY9"
2,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,"A0A1X7RBG6, Q7GXY9"
3,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,"A0A1X7RBG2, U5YWV7"
4,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,"A0A1X7RBG2, U5YWV7"
...,...,...,...,...,...,...
119506,ENSG00000143569,ENST00000361546,ENSP00000355343,Q14157-2,Q14157-1,
119509,ENSG00000143575,ENST00000457918,ENSP00000411448,O00165-5,O00165-1,A0A0S2Z565
119510,ENSG00000143575,ENST00000483970,ENSP00000435088,O00165-2,O00165-1,
119511,ENSG00000143575,ENST00000447768,ENSP00000403848,O00165-6,O00165-1,


In [43]:
au2 = au2.drop_duplicates('uniprot_trembl_id')

In [44]:
au2[(~au2.duplicated()) | (df['col'].isnull())]

,Gene stable ID,Transcript stable ID,Protein stable ID,UniProtKB isoform ID,UniProtKB/Swiss-Prot ID,UniProtKB/TrEMBL ID,uniprot_trembl_id
0,ENSG00000198888,ENST00000361390,ENSP00000354687,P03886-1,P03886-1,U5Z754,U5Z754
1,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,A0A1X7RBG6,"A0A1X7RBG6, Q7GXY9"
3,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,U5YWV7,"A0A1X7RBG2, U5YWV7"
5,ENSG00000198712,ENST00000361739,ENSP00000354876,P00403-1,P00403-1,U5Z487,"A0A1X7RBF7, U5Z487"
7,ENSG00000228253,ENST00000361851,ENSP00000355265,P03928-1,P03928-1,U5YV54,"A0A1X7RCP4, U5YV54"
...,...,...,...,...,...,...,...
119470,ENSG00000143549,ENST00000368533,ENSP00000357521,P06753-2,P06753-1,A0A0S2Z4G4,A0A0S2Z4G4
119471,ENSG00000143549,ENST00000330188,ENSP00000339035,P06753-5,P06753-1,A0A0S2Z4I4,A0A0S2Z4I4
119492,ENSG00000143612,ENST00000350592,ENSP00000271925,Q9BWL3-2,Q9BWL3-1,Q09GN0,Q09GN0
119509,ENSG00000143575,ENST00000457918,ENSP00000411448,O00165-5,O00165-1,A0A0S2Z565,A0A0S2Z565


In [41]:
au2[au2.duplicated('Transcript stable ID', keep=False)]

,Gene stable ID,Transcript stable ID,Protein stable ID,UniProtKB isoform ID,UniProtKB/Swiss-Prot ID,UniProtKB/TrEMBL ID,uniprot_trembl_id
1,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,A0A1X7RBG6,"A0A1X7RBG6, Q7GXY9"
2,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,Q7GXY9,"A0A1X7RBG6, Q7GXY9"
3,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,U5YWV7,"A0A1X7RBG2, U5YWV7"
4,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,A0A1X7RBG2,"A0A1X7RBG2, U5YWV7"
5,ENSG00000198712,ENST00000361739,ENSP00000354876,P00403-1,P00403-1,U5Z487,"A0A1X7RBF7, U5Z487"
...,...,...,...,...,...,...,...
116467,ENSG00000167085,ENST00000419140,ENSP00000393320,P35232-1,P35232-1,A8K401,"A8K401, C9JW96"
116469,ENSG00000167085,ENST00000504124,ENSP00000426433,P35232-1,P35232-1,E9PCW0,"A8K401, E9PCW0"
116470,ENSG00000167085,ENST00000504124,ENSP00000426433,P35232-1,P35232-1,A8K401,"A8K401, E9PCW0"
116473,ENSG00000167085,ENST00000512041,ENSP00000422182,P35232-1,P35232-1,E7ESE2,"A8K401, E7ESE2"


In [31]:
ae = pd.read_csv('../missing-crispr-apms/data/processed/ensembl_annotation_trs_uniprot_20220429.csv')

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14780\1863431825.py:1: DtypeWarning: Columns (0,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  ae = pd.read_csv('../missing-crispr-apms/data/processed/ensembl_annotation_trs_uniprot_20220429.csv')


In [38]:
ae[~ae['uniprot_base'].isna()].drop_duplicates('uniprot_isoform')

,ensembl_gene_name,ensembl_gene_id,ensembl_trs_id,ensembl_is_canonical,trs_type,trs_length_bp,ensembl_protein_id,uniprot_base,uniprot_isoform
45,A1BG,ENSG00000121410,ENST00000263100,True,protein_coding,3382,ENSP00000263100,P04217,P04217-1
59,A1CF,ENSG00000148584,ENST00000373993,False,protein_coding,1997,ENSP00000363105,Q9NQ94,Q9NQ94-1
60,A1CF,ENSG00000148584,ENST00000374001,False,protein_coding,9221,ENSP00000363113,Q9NQ94,Q9NQ94-2
63,A1CF,ENSG00000148584,ENST00000373995,False,protein_coding,2211,ENSP00000363107,Q9NQ94,Q9NQ94-4
69,A2M,ENSG00000175899,ENST00000318602,True,protein_coding,4610,ENSP00000323929,P01023,P01023-1
...,...,...,...,...,...,...,...,...,...
234646,NaN,ENSG00000285133,ENST00000642841,True,protein_coding,105,ENSP00000494177,P0DPQ6,P0DPQ6-1
234654,NaN,ENSG00000283886,ENST00000622735,True,protein_coding,1888,ENSP00000499803,Q8NFD4,Q8NFD4-1
234655,NaN,ENSG00000288631,ENST00000674825,True,protein_coding,1641,ENSP00000502009,Q8NG57,Q8NG57-1
234656,NaN,ENSG00000273590,ENST00000612624,False,protein_coding,1021,ENSP00000483089,Q8TCY0,Q8TCY0-1


In [53]:
au[au.duplicated(['Transcript stable ID', 'UniProtKB isoform ID', 'uniprot_trembl_id'], keep=False)]

,Gene stable ID,Transcript stable ID,Protein stable ID,UniProtKB isoform ID,UniProtKB/Swiss-Prot ID,UniProtKB/TrEMBL ID,uniprot_trembl_id
1,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,A0A1X7RBG6,"A0A1X7RBG6, Q7GXY9"
2,ENSG00000198763,ENST00000361453,ENSP00000355046,P03891-1,P03891-1,Q7GXY9,"A0A1X7RBG6, Q7GXY9"
3,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,U5YWV7,"A0A1X7RBG2, U5YWV7"
4,ENSG00000198804,ENST00000361624,ENSP00000354499,P00395-1,P00395-1,A0A1X7RBG2,"A0A1X7RBG2, U5YWV7"
5,ENSG00000198712,ENST00000361739,ENSP00000354876,P00403-1,P00403-1,U5Z487,"A0A1X7RBF7, U5Z487"
...,...,...,...,...,...,...,...
118450,ENSG00000143153,ENST00000494797,ENSP00000477015,NaN,NaN,B7Z9S8,"B7Z9S8, V9GYR2"
118767,ENSG00000173372,ENST00000438241,ENSP00000416841,NaN,NaN,X6RLJ0,"A0A024RAG6, X6RLJ0"
118768,ENSG00000173372,ENST00000438241,ENSP00000416841,NaN,NaN,A0A024RAG6,"A0A024RAG6, X6RLJ0"
118843,ENSG00000070831,ENST00000411827,ENSP00000398327,NaN,NaN,Q5JYX0,"A0A024RAA5, Q5JYX0"


In [54]:
au2 = au.drop_duplicates(['Transcript stable ID'])

In [58]:
au2['uniprot_trembl_id'] = au2['uniprot_trembl_id'].replace('', np.nan)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14780\3010929671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  au2['uniprot_trembl_id'] = au2['uniprot_trembl_id'].replace('', np.nan)


In [60]:
au2 = au2.drop(['Gene stable ID', 'Protein stable ID', 'UniProtKB/TrEMBL ID'], axis=1)

In [62]:
au2 = au2.rename({'Transcript stable ID': 'ensembl_trs_id', 'UniProtKB isoform ID': 'uniprot_isoform_id', 'UniProtKB/Swiss-Prot ID': 'uniprot_base_id'}, axis=1)

In [65]:
au2['uniprot_base_id'] = au2['uniprot_base_id'].map(lambda x: x[:x.index('-')] if x is not np.nan else x)

In [68]:
au2 = au2[['ensembl_trs_id', 'uniprot_base_id', 'uniprot_isoform_id', 'uniprot_trembl_id']]

In [69]:
au2 = au2.set_index('ensembl_trs_id', drop=True)

In [70]:
au2

,uniprot_base_id,uniprot_isoform_id,uniprot_trembl_id
ensembl_trs_id,,,
ENST00000361390,P03886,P03886-1,U5Z754
ENST00000361453,P03891,P03891-1,"A0A1X7RBG6, Q7GXY9"
ENST00000361624,P00395,P00395-1,"A0A1X7RBG2, U5YWV7"
ENST00000361739,P00403,P00403-1,"A0A1X7RBF7, U5Z487"
ENST00000361851,P03928,P03928-1,"A0A1X7RCP4, U5YV54"
...,...,...,...
ENST00000483970,O00165,O00165-2,NaN
ENST00000447768,O00165,O00165-6,NaN
ENST00000328703,O00165,O00165-1,A0A0S2Z591


In [73]:
ag2 = ag.join(au2, on='ensembl_trs_id')

In [76]:
ap.drop_duplicates('Transcript stable ID')

,Transcript stable ID,UCSC Stable ID
0,ENST00000387314,NaN
1,ENST00000389680,NaN
2,ENST00000387342,NaN
3,ENST00000387347,NaN
4,ENST00000386347,NaN
...,...,...
268955,ENST00000435087,uc001fes.6
268956,ENST00000532105,uc057lhl.1
268957,ENST00000471326,uc057lhm.1
268958,ENST00000492550,uc057lhn.1


In [78]:
ap[ap.duplicated('Transcript stable ID', keep=False)]

,Transcript stable ID,UCSC Stable ID
24877,ENST00000639527,uc062fgq.2
24878,ENST00000639527,uc285tju.1
26270,ENST00000437180,uc288ouf.1
26271,ENST00000437180,uc010gmv.3
29720,ENST00000395744,uc003bla.3
...,...,...
267979,ENST00000696310,uc057iba.1
268219,ENST00000664805,uc288gqr.1
268220,ENST00000664805,uc288azg.1
268692,ENST00000664374,uc287snu.1


In [80]:
ag2.to_csv('data/annotation.csv', index=False)

In [17]:
ae = pd.read_csv('data/raw/ensembl_exons.txt')

In [83]:
ag[ag['gene_type'] == 'protein_coding']

,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_id,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end,in_gencode_basic,appris_annotation
5,MT,ENSG00000198888,MT-ND1,protein_coding,1,3307,4262,ENST00000361390,MT-ND1-201,protein_coding,True,tslNA,956,3307,4262,True,principal1
9,MT,ENSG00000198763,MT-ND2,protein_coding,1,4470,5511,ENST00000361453,MT-ND2-201,protein_coding,True,tslNA,1042,4470,5511,True,principal1
15,MT,ENSG00000198804,MT-CO1,protein_coding,1,5904,7445,ENST00000361624,MT-CO1-201,protein_coding,True,tslNA,1542,5904,7445,True,principal1
18,MT,ENSG00000198712,MT-CO2,protein_coding,1,7586,8269,ENST00000361739,MT-CO2-201,protein_coding,True,tslNA,684,7586,8269,True,principal1
20,MT,ENSG00000228253,MT-ATP8,protein_coding,1,8366,8572,ENST00000361851,MT-ATP8-201,protein_coding,True,tslNA,207,8366,8572,True,principal1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268296,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000435087,HAX1-202,protein_coding,False,tsl3,908,154272668,154275701,True,NaN
268297,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000532105,HAX1-211,protein_coding,False,tsl3,771,154272668,154275839,True,NaN
268298,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000471326,HAX1-206,retained_intron,False,tsl2,1170,154272868,154275484,False,NaN
268299,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000492550,HAX1-209,retained_intron,False,tsl5,733,154273869,154275873,False,NaN


In [18]:
ae2 = ae.rename({'Gene stable ID': 'ensembl_gene_id', 'Transcript stable ID': 'ensembl_trs_id', '5\' UTR start': '5utr_start', 
                '5\' UTR end': '5utr_end', '3\' UTR start': '3utr_start', '3\' UTR end': '3utr_end', 'Exon region start (bp)': 'exon_genomic_start', 'Exon region end (bp)': 'exon_genomic_end',
                'Exon stable ID': 'ensembl_exon_id', 'Constitutive exon': 'exon_is_constitutive', 'Genomic coding start': 'cds_genomic_start', 'Genomic coding end': 'cds_genomic_end',
                'CDS Length': 'cds_length_bp', 'CDS end': 'cds_internal_end', 'CDS start': 'cds_internal_start'}, axis=1)

In [19]:
ae2

,ensembl_gene_id,ensembl_trs_id,5utr_start,5utr_end,3utr_start,3utr_end,cds_length_bp,exon_genomic_start,exon_genomic_end,exon_is_constitutive,ensembl_exon_id,cds_genomic_start,cds_genomic_end,cds_internal_end,cds_internal_start
0,ENSG00000210049,ENST00000387314,NaN,NaN,NaN,NaN,NaN,577,647,1,ENSE00001544501,NaN,NaN,NaN,NaN
1,ENSG00000211459,ENST00000389680,NaN,NaN,NaN,NaN,NaN,648,1601,1,ENSE00001544499,NaN,NaN,NaN,NaN
2,ENSG00000210077,ENST00000387342,NaN,NaN,NaN,NaN,NaN,1602,1670,1,ENSE00001544498,NaN,NaN,NaN,NaN
3,ENSG00000210082,ENST00000387347,NaN,NaN,NaN,NaN,NaN,1671,3229,1,ENSE00001544497,NaN,NaN,NaN,NaN
4,ENSG00000209082,ENST00000386347,NaN,NaN,NaN,NaN,NaN,3230,3304,1,ENSE00002006242,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697652,ENSG00000143575,ENST00000492550,NaN,NaN,NaN,NaN,NaN,154275616,154275873,0,ENSE00003563146,NaN,NaN,NaN,NaN
1697653,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154275393,154275483,0,ENSE00003569654,NaN,NaN,NaN,NaN
1697654,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154275616,154275875,0,ENSE00003660144,NaN,NaN,NaN,NaN
1697655,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154273898,154273961,0,ENSE00001948674,NaN,NaN,NaN,NaN


In [94]:
ag2['cds_length_bp'] = ag['ensembl_trs_id'].map(ae2.drop_duplicates('ensembl_trs_id').set_index('ensembl_trs_id')['cds_length_bp'])

In [95]:
ag2['protein_length_aa'] = ag['cds_length_bp'].map(lambda x: (x - 3) / 3 if x is not np.nan else np.nan)

In [101]:
ag2['cds_length_bp'] = ag2['cds_length_bp'].astype(float)

In [108]:
ag2['uniprot_trembl_id'].dtype

dtype('O')

In [113]:
ag2.iloc[:, :15]

,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_id,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end
0,MT,ENSG00000210049,MT-TF,Mt_tRNA,1,577,647,ENST00000387314,MT-TF-201,Mt_tRNA,True,tslNA,71,577,647
1,MT,ENSG00000211459,MT-RNR1,Mt_rRNA,1,648,1601,ENST00000389680,MT-RNR1-201,Mt_rRNA,True,tslNA,954,648,1601
2,MT,ENSG00000210077,MT-TV,Mt_tRNA,1,1602,1670,ENST00000387342,MT-TV-201,Mt_tRNA,True,tslNA,69,1602,1670
3,MT,ENSG00000210082,MT-RNR2,Mt_rRNA,1,1671,3229,ENST00000387347,MT-RNR2-201,Mt_rRNA,True,tslNA,1559,1671,3229
4,MT,ENSG00000209082,MT-TL1,Mt_tRNA,1,3230,3304,ENST00000386347,MT-TL1-201,Mt_tRNA,True,tslNA,75,3230,3304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268296,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000435087,HAX1-202,protein_coding,False,tsl3,908,154272668,154275701
268297,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000532105,HAX1-211,protein_coding,False,tsl3,771,154272668,154275839
268298,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000471326,HAX1-206,retained_intron,False,tsl2,1170,154272868,154275484
268299,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000492550,HAX1-209,retained_intron,False,tsl5,733,154273869,154275873


In [114]:
ag

,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_id,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end,in_gencode_basic,appris_annotation,cds_length_bp,protein_length_aa
0,MT,ENSG00000210049,MT-TF,Mt_tRNA,1,577,647,ENST00000387314,MT-TF-201,Mt_tRNA,True,tslNA,71,577,647,True,NaN,NaN,NaN
1,MT,ENSG00000211459,MT-RNR1,Mt_rRNA,1,648,1601,ENST00000389680,MT-RNR1-201,Mt_rRNA,True,tslNA,954,648,1601,True,NaN,NaN,NaN
2,MT,ENSG00000210077,MT-TV,Mt_tRNA,1,1602,1670,ENST00000387342,MT-TV-201,Mt_tRNA,True,tslNA,69,1602,1670,True,NaN,NaN,NaN
3,MT,ENSG00000210082,MT-RNR2,Mt_rRNA,1,1671,3229,ENST00000387347,MT-RNR2-201,Mt_rRNA,True,tslNA,1559,1671,3229,True,NaN,NaN,NaN
4,MT,ENSG00000209082,MT-TL1,Mt_tRNA,1,3230,3304,ENST00000386347,MT-TL1-201,Mt_tRNA,True,tslNA,75,3230,3304,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268296,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000435087,HAX1-202,protein_coding,False,tsl3,908,154272668,154275701,True,NaN,852.0,283.0
268297,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000532105,HAX1-211,protein_coding,False,tsl3,771,154272668,154275839,True,NaN,456.0,151.0
268298,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000471326,HAX1-206,retained_intron,False,tsl2,1170,154272868,154275484,False,NaN,NaN,NaN
268299,1,ENSG00000143575,HAX1,protein_coding,1,154272589,154275875,ENST00000492550,HAX1-209,retained_intron,False,tsl5,733,154273869,154275873,False,NaN,NaN,NaN


In [116]:
ag0 = pd.read_csv('data/raw/ensembl_general.txt')

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14780\3563040682.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ag0 = pd.read_csv('data/raw/ensembl_general.txt')


In [118]:
ag2['ensembl_protein_id'] = ag2['ensembl_trs_id'].map(ag0.drop_duplicates('Transcript stable ID').set_index('Transcript stable ID')['Protein stable ID'])

In [120]:
ag2.columns

Index(['chromosome_or_scaffold', 'ensembl_gene_id', 'ensembl_gene_name',
       'gene_type', 'strand', 'gene_start', 'gene_end', 'ensembl_trs_id',
       'ensembl_trs_name', 'trs_type', 'is_ensembl_canonical',
       'trs_support_level', 'trs_length_bp', 'trs_start', 'trs_end',
       'in_gencode_basic', 'appris_annotation', 'uniprot_base_id',
       'uniprot_isoform_id', 'uniprot_trembl_id', 'cds_length_bp',
       'protein_length_aa', 'ensembl_protein_id'],
      dtype='object')

In [7]:
ag2 = pd.read_csv('data/ensembl_annotation_main.csv')

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14808\109610476.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ag2 = pd.read_csv('data/ensembl_annotation_main.csv')


In [14]:
ae3

,ensembl_gene_id,ensembl_trs_id,5utr_start,5utr_end,3utr_end,3utr_end,cds_length_bp,exon_genomic_start,exon_genomic_end,exon_is_constitutive,ensembl_exon_id,cds_genomic_start,cds_genomic_end,cds_internal_end,cds_internal_start
0,ENSG00000210049,ENST00000387314,NaN,NaN,NaN,NaN,NaN,577,647,1,ENSE00001544501,NaN,NaN,NaN,NaN
1,ENSG00000211459,ENST00000389680,NaN,NaN,NaN,NaN,NaN,648,1601,1,ENSE00001544499,NaN,NaN,NaN,NaN
2,ENSG00000210077,ENST00000387342,NaN,NaN,NaN,NaN,NaN,1602,1670,1,ENSE00001544498,NaN,NaN,NaN,NaN
3,ENSG00000210082,ENST00000387347,NaN,NaN,NaN,NaN,NaN,1671,3229,1,ENSE00001544497,NaN,NaN,NaN,NaN
4,ENSG00000209082,ENST00000386347,NaN,NaN,NaN,NaN,NaN,3230,3304,1,ENSE00002006242,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697652,ENSG00000143575,ENST00000492550,NaN,NaN,NaN,NaN,NaN,154275616,154275873,0,ENSE00003563146,NaN,NaN,NaN,NaN
1697653,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154275393,154275483,0,ENSE00003569654,NaN,NaN,NaN,NaN
1697654,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154275616,154275875,0,ENSE00003660144,NaN,NaN,NaN,NaN
1697655,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154273898,154273961,0,ENSE00001948674,NaN,NaN,NaN,NaN


In [123]:
ae2['gene_start'] = ae2['ensembl_trs_id'].map(ag2.set_index('ensembl_trs_id')['gene_start'])

,Gene stable ID,Transcript stable ID,5' UTR start,5' UTR end,3' UTR start,3' UTR end,CDS Length,Exon region start (bp),Exon region end (bp),Constitutive exon,Exon stable ID,Genomic coding start,Genomic coding end,CDS end,CDS start
0,ENSG00000210049,ENST00000387314,NaN,NaN,NaN,NaN,NaN,577,647,1,ENSE00001544501,NaN,NaN,NaN,NaN
1,ENSG00000211459,ENST00000389680,NaN,NaN,NaN,NaN,NaN,648,1601,1,ENSE00001544499,NaN,NaN,NaN,NaN
2,ENSG00000210077,ENST00000387342,NaN,NaN,NaN,NaN,NaN,1602,1670,1,ENSE00001544498,NaN,NaN,NaN,NaN
3,ENSG00000210082,ENST00000387347,NaN,NaN,NaN,NaN,NaN,1671,3229,1,ENSE00001544497,NaN,NaN,NaN,NaN
4,ENSG00000209082,ENST00000386347,NaN,NaN,NaN,NaN,NaN,3230,3304,1,ENSE00002006242,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697652,ENSG00000143575,ENST00000492550,NaN,NaN,NaN,NaN,NaN,154275616,154275873,0,ENSE00003563146,NaN,NaN,NaN,NaN
1697653,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154275393,154275483,0,ENSE00003569654,NaN,NaN,NaN,NaN
1697654,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154275616,154275875,0,ENSE00003660144,NaN,NaN,NaN,NaN
1697655,ENSG00000143575,ENST00000459914,NaN,NaN,NaN,NaN,NaN,154273898,154273961,0,ENSE00001948674,NaN,NaN,NaN,NaN


In [20]:
ae3 = ae2.copy()

In [21]:
ag2['5utr_start'] = ag2[ag2['strand'] == 1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['5utr_start'].min())
ag2['5utr_start_r'] = ag2[ag2['strand'] == -1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['5utr_start'].max())
ag2['5utr_start'] = ag2['5utr_start'].combine_first(ag2['5utr_start_r'])
ag2 = ag2.drop('5utr_start_r', axis=1)

In [22]:
ag2['5utr_end'] = ag2[ag2['strand'] == 1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['5utr_end'].max())
ag2['5utr_end_r'] = ag2[ag2['strand'] == -1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['5utr_end'].min())
ag2['5utr_end'] = ag2['5utr_end'].combine_first(ag2['5utr_end_r'])
ag2 = ag2.drop('5utr_end_r', axis=1)

In [23]:
ag2['3utr_start'] = ag2[ag2['strand'] == 1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['3utr_start'].min())
ag2['3utr_start_r'] = ag2[ag2['strand'] == -1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['3utr_start'].max())
ag2['3utr_start'] = ag2['3utr_start'].combine_first(ag2['3utr_start_r'])
ag2 = ag2.drop('3utr_start_r', axis=1)

In [24]:
ag2['3utr_end'] = ag2[ag2['strand'] == 1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['3utr_end'].max())
ag2['3utr_end_r'] = ag2[ag2['strand'] == -1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['3utr_end'].min())
ag2['3utr_end'] = ag2['3utr_end'].combine_first(ag2['3utr_end_r'])
ag2 = ag2.drop('3utr_end_r', axis=1)

In [25]:
ag2['cds_start'] = ag2[ag2['strand'] == 1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['cds_genomic_start'].min())
ag2['cds_start_r'] = ag2[ag2['strand'] == -1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['cds_genomic_start'].max())
ag2['cds_start'] = ag2['cds_start'].combine_first(ag2['cds_start_r'])
ag2 = ag2.drop('cds_start_r', axis=1)

In [26]:
ag2['cds_end'] = ag2[ag2['strand'] == 1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['cds_genomic_end'].max())
ag2['cds_end_r'] = ag2[ag2['strand'] == -1]['ensembl_trs_id'].map(ae3.groupby('ensembl_trs_id')['cds_genomic_end'].min())
ag2['cds_end'] = ag2['cds_end'].combine_first(ag2['cds_end_r'])
ag2 = ag2.drop('cds_end_r', axis=1)

In [157]:
ag2[ag2['ensembl_gene_name'] == 'CNPY1']

,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_id,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end,in_gencode_basic,appris_annotation,uniprot_base_id,uniprot_isoform_id,uniprot_trembl_id,cds_length_bp,protein_length_aa,ensembl_protein_id,5utr_start,5utr_end,3utr_start,3utr_end,cds_start,cds_end
219671,7,ENSG00000146910,CNPY1,protein_coding,-1,155474206,155555450,ENST00000406197,CNPY1-202,protein_coding,False,tsl5,560,155474206,155533838,True,NaN,Q3B7I2,Q3B7I2-1,NaN,279.0,92.0,ENSP00000384514,155533760.0,155509097.0,155503021.0,155474300.0,155508894.0,155503105.0
219672,7,ENSG00000146910,CNPY1,protein_coding,-1,155474206,155555450,ENST00000682997,CNPY1-205,protein_coding,False,tslNA,2575,155501129,155510162,True,NaN,NaN,NaN,NaN,354.0,117.0,ENSP00000507260,155509736.0,155510162.0,155503021.0,155502875.0,155509721.0,155503105.0
219673,7,ENSG00000146910,CNPY1,protein_coding,-1,155474206,155555450,ENST00000636446,CNPY1-204,protein_coding,True,tsl5,2522,155501129,155546559,True,principal1,NaN,NaN,A0A1B0GVE0,438.0,145.0,ENSP00000490477,155546429.0,155545943.0,155501129.0,155503067.0,155545831.0,155503105.0
219674,7,ENSG00000146910,CNPY1,protein_coding,-1,155474206,155555450,ENST00000321736,CNPY1-201,protein_coding,False,tsl1,2378,155501132,155533862,True,NaN,Q3B7I2,Q3B7I2-1,NaN,279.0,92.0,ENSP00000317439,155533760.0,155509097.0,155501132.0,155503067.0,155508894.0,155503105.0
219675,7,ENSG00000146910,CNPY1,protein_coding,-1,155474206,155555450,ENST00000636372,CNPY1-203,protein_coding,False,tsl5,2259,155502014,155555450,True,principal1,NaN,NaN,A0A1B0GUD4,438.0,145.0,ENSP00000490060,155554553.0,155545943.0,155503021.0,155502875.0,155545831.0,155503105.0


In [28]:
ae3.to_csv('data/ens_exons.csv', index=False)

In [30]:
ag2.to_csv('data/ens_main.csv', index=False)

In [36]:
ens_exons = ae3.drop(['ensembl_gene_id', 'cds_length_bp', 'exon_is_constitutive', 'cds_internal_start', 'cds_internal_end'], axis=1).copy(); ens_exons

,ensembl_trs_id,5utr_start,5utr_end,3utr_start,3utr_end,exon_genomic_start,exon_genomic_end,ensembl_exon_id,cds_genomic_start,cds_genomic_end
0,ENST00000387314,NaN,NaN,NaN,NaN,577,647,ENSE00001544501,NaN,NaN
1,ENST00000389680,NaN,NaN,NaN,NaN,648,1601,ENSE00001544499,NaN,NaN
2,ENST00000387342,NaN,NaN,NaN,NaN,1602,1670,ENSE00001544498,NaN,NaN
3,ENST00000387347,NaN,NaN,NaN,NaN,1671,3229,ENSE00001544497,NaN,NaN
4,ENST00000386347,NaN,NaN,NaN,NaN,3230,3304,ENSE00002006242,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1697652,ENST00000492550,NaN,NaN,NaN,NaN,154275616,154275873,ENSE00003563146,NaN,NaN
1697653,ENST00000459914,NaN,NaN,NaN,NaN,154275393,154275483,ENSE00003569654,NaN,NaN
1697654,ENST00000459914,NaN,NaN,NaN,NaN,154275616,154275875,ENSE00003660144,NaN,NaN
1697655,ENST00000459914,NaN,NaN,NaN,NaN,154273898,154273961,ENSE00001948674,NaN,NaN


In [37]:
ens_exons = ens_exons.rename({'exon_genomic_start':'exon_start', 'exon_genomic_end':'exon_end', 'cds_genomic_start':'cds_start', 'cds_genomic_end': 'cds_end'}, axis=1)
ens_exons = ens_exons[['ensembl_trs_id', 'ensembl_exon_id', '5utr_start', '5utr_end', '3utr_start', '3utr_end','exon_start', 'exon_end','cds_start', 'cds_end']]

In [40]:
ens_exons.iloc[:, 2:] = ens_exons.iloc[:, 2:].astype('Int64')

In [47]:
ens_main = ag2[['chromosome_or_scaffold', 'ensembl_gene_id', 'ensembl_gene_name',
                'gene_type', 'strand', 'gene_start', 'gene_end', 'ensembl_trs_id', 'trs_type', 'is_ensembl_canonical',
                'trs_support_level', 'trs_start', 'trs_end',
                'in_gencode_basic', 'appris_annotation', 'uniprot_base_id',
                'uniprot_isoform_id', 'uniprot_trembl_id', '5utr_start', '5utr_end',
                '3utr_start', '3utr_end', 'cds_start', 'cds_end']]

In [49]:
ens_main.loc[:, ['5utr_start', '5utr_end', '3utr_start', '3utr_end', 'cds_start', 'cds_end']] = ens_main.loc[:, ['5utr_start', '5utr_end', '3utr_start', '3utr_end', 'cds_start', 'cds_end']].astype('Int64')

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14808\1221152612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_main.loc[:, ['5utr_start', '5utr_end', '3utr_start', '3utr_end', 'cds_start', 'cds_end']] = ens_main.loc[:, ['5utr_start', '5utr_end', '3utr_start', '3utr_end', 'cds_start', 'cds_end']].astype('Int64')


In [53]:
ens_main.to_csv('data/ens_main_int.csv', index=False)
ens_exons.to_csv('data/ens_exons_int.csv', index=False)

In [277]:
gene = 'TEF'
gene_view = ag2[ag2['ensembl_gene_name'] == 'TEF']

gene_start = gene_view.loc[:, 'gene_start'].iloc[0]
gene_end = gene_view.loc[:, 'gene_end'].iloc[0]

In [278]:
cds_pos_arr = np.zeros((len(gene_view['trs_type'] == 'protein-coding'), gene_end - gene_start + 1), dtype=int)
for i,j in enumerate(gene_view[gene_view['trs_type'] == 'protein_coding']['ensembl_trs_id'].to_list()):
    cds_pos_arr[i, (gene_view.iloc[i, [-2,-1]].astype(int) - gene_start).to_list()] = 1

In [279]:
seq_pos_arr = np.zeros((len(gene_view), gene_end - gene_start + 1), dtype=int)
for i,j in enumerate(gene_view['ensembl_trs_id'].to_list()):
    exon_view = ae3[ae3['ensembl_trs_id'] == j].loc[:, ['exon_genomic_start', 'exon_genomic_end']].copy()
    exon_ranges = [np.arange(i[0], i[1]) for i in zip((exon_view.loc[:, "exon_genomic_start"] - gene_start).to_list(), (exon_view.loc[:, "exon_genomic_end"] - gene_start + 1).to_list())]
    seq_pos_arr[i, np.hstack(exon_ranges)] = 1

In [280]:
cds_mask = np.where((cds_pos_arr == 1).any(0))
cds_pos_arr = cds_pos_arr[:, cds_mask[0]]
seq_pos_arr = seq_pos_arr[:, cds_mask[0]].sum(axis=0)

In [282]:
#Min solution
trs_w_unique_cds_ends = np.where(cds_pos_arr - seq_pos_arr == 0)[0]
if len(trs_w_unique_cds_ends) >= 2:
    # Propagate solution...
    pass

In [340]:
s = range(len(cds_pos_arr))
solutions = []

for indices in sorted(list(chain.from_iterable(combinations(s, r) for r in range(2, len(s)+1))), key=len, reverse=True):
    indices = list(indices)
    cds_sum = cds_pos_arr[indices, :].sum(axis=0)
    c = Counter(cds_sum)
    if c[len(indices)] == 1 and c[1] == len(indices):
        mask = np.where(cds_sum >= 1)[0]
        if not (cds_sum[mask] - seq_pos_arr[mask]).any():
            solutions.append(indices)

solutions_ids = []
for i in solutions:
    solutions_ids.append(gene_view.iloc[i, :]['ensembl_trs_id'].to_list())

In [ ]:
all_pc_genes = ag3[(ag3['gene_type'] == 'protein_coding') & (ag2['chromosome_or_scaffold'].isin(list(range(1,23)) + ['X', 'Y']))]['ensembl_gene_name'].dropna().unique()

hits = {}

for x, gene in enumerate(all_pc_genes):
    try:
        hits[gene] = []

        gene_view = ag2[ag2['ensembl_gene_name'] == gene]

        gene_start = gene_view.loc[:, 'gene_start'].iloc[0]
        gene_end = gene_view.loc[:, 'gene_end'].iloc[0]

        cds_pos_arr = np.zeros((len(gene_view['trs_type'] == 'protein_coding'), gene_end - gene_start + 1), dtype=int)

        for i,j in enumerate(gene_view[gene_view['trs_type'] == 'protein_coding']['ensembl_trs_id'].to_list()):
            cds_pos_arr[i, (gene_view.iloc[i, [-2,-1]].astype(int) - gene_start).to_list()] = 1

        seq_pos_arr = np.zeros((len(gene_view), gene_end - gene_start + 1), dtype=int)

        for i,j in enumerate(gene_view['ensembl_trs_id'].to_list()):
            exon_view = ae3[ae3['ensembl_trs_id'] == j].loc[:, ['exon_genomic_start', 'exon_genomic_end']].copy()
            exon_ranges = [np.arange(i[0], i[1]) for i in zip((exon_view.loc[:, "exon_genomic_start"] - gene_start).to_list(), (exon_view.loc[:, "exon_genomic_end"] - gene_start + 1).to_list())]
            seq_pos_arr[i, np.hstack(exon_ranges)] = 1

        cds_mask = np.where((cds_pos_arr == 1).any(0))
        cds_pos_arr = cds_pos_arr[:, cds_mask[0]]
        seq_pos_arr = seq_pos_arr[:, cds_mask[0]].sum(axis=0)

        s = range(len(cds_pos_arr))

        solutions = []

        for indices in sorted(list(chain.from_iterable(combinations(s, r) for r in range(2, len(s)+1))), key=len, reverse=True):
            indices = list(indices)
            cds_sum = cds_pos_arr[indices, :].sum(axis=0)
            c = Counter(cds_sum)
            if c[len(indices)] == 1 and c[1] == len(indices):
                mask = np.where(cds_sum >= 1)[0]
                if not (cds_sum[mask] - seq_pos_arr[mask]).any():
                    solutions.append(indices)

        solutions_ids = []
        for i in solutions:
            solutions_ids.append(gene_view.iloc[i, :]['ensembl_trs_id'].to_list())

        hits[gene].extend(solutions_ids)
    except:
        print(gene)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_14780\649904824.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_pc_genes = ag3[(ag3['gene_type'] == 'protein_coding') & (ag2['chromosome_or_scaffold'].isin(list(range(1,23)) + ['X', 'Y']))]['ensembl_gene_name'].dropna().unique()


0
0
USP9Y
0
0
1
EIF1AY
0
1
2
BPY2
0
0
1
0
0
1
0
1
2
PCDH11Y
0
RBMY1J
0
1
0
1
0
1
2
BPY2B
0
1
HSFY2
0
KDM5D
0
1
2
0
1
0
1
RBMY1F
0
0
1
0
1
BPY2C
0
1
2
3
4
5
6
7
0
0
0
1
0
1
2
3
ZFY
0
1
2
DDX3Y
0
1
2
UTY
0
0
0
1
2
0
1
2
DAZ1
0
1
0
1
2
3
0
1
2
0
1
2
0
1
2
0
1
2
TSPY8
0
1
2
0
1
0
1
NLGN4Y
0
0
0
1
0
1
2
0
1
PPP2R3B
0
0
0
1
2
0
1
0
1
2
3
0
0
1
2
0
1
2
3
ASMT
0
GTPBP6
0
1
2
DHRSX
0
1
SHROOM2
0
0
1
2
ASMTL
0
1
2
0
1
2
3
4
FAM9B
0
1
2
0
NLGN4X
0
1
FAM9C
0
1
0
1
SRPX
0
0
1
0
0
0
1
0
1
0
1
2
3
CNKSR2
0
0
1
0
1
TXLNG
0
0
1
2
3
0
1
2
3
4
5
6
7
ADGRG2
0
1
2
3
4
5
0
1
2
CTPS2
0
0
0
0
MBTPS2
0
1
0
1
0
0
1
2
SCML2
0
1
0
0
EFHC2
0
1
2
0
0
0
1
2
3
4
5
6
0
0
1
2
3
4
0
1
0
0
1
0
0
0
1
2
3
4
5
6
7
8
9
0
1
0
0
1
2
3
0
1
2
3
CFAP47
0
0
1
0
0
1
2
0
1
2
AKAP4
0
0
1
2
3
0
1
2
3
FOXR2
0
1
SLC9A7
0
DGKK
0
PIR
0
1
2
3
EDA2R
0
1
CYSLTR1
0
1
2
3
ATP6AP2
0
1
SLC16A2
0
FGD1
0
1
2
MAGED2
0
KLF8
0
XAGE2
0
0
1
2
0
1
2
3
4
5
HDAC8
0
1
UPRT
0
1
2
3
HDX
0
1
0
0
1
0
0
1
0
1
2
3
EDA
0
1
CHM
0
1
2
3
4
5
6
0
1
RPS6KA6
0
1
0
1
0


In [354]:
all_pc_genes

array(['KIR3DL1', 'KIR2DL1', 'KIR2DS5', ..., 'C1orf43', 'UBAP2L', 'HAX1'],
      dtype=object)

In [360]:
ag3[ag3['ensembl_gene_name'] == 'KIR3DL1']

,chromosome_or_scaffold,ensembl_gene_id,ensembl_gene_name,gene_type,strand,gene_start,gene_end,ensembl_trs_id,ensembl_trs_name,trs_type,is_ensembl_canonical,trs_support_level,trs_length_bp,trs_start,trs_end,in_gencode_basic,appris_annotation,uniprot_base_id,uniprot_isoform_id,uniprot_trembl_id,cds_length_bp,protein_length_aa,ensembl_protein_id,5utr_start,5utr_end,3utr_start,3utr_end,cds_start,cds_end
41,CHR_HSCHR19KIR_ABC08_AB_HAP_C_P_CTG3_1,ENSG00000276379,KIR3DL1,protein_coding,1,54816496,54820397,ENST00000612571,KIR3DL1-227,protein_coding,True,tsl1,671,54816496,54820397,True,NaN,NaN,NaN,A0A0G2JQ40,657.0,218.0,ENSP00000484895,54816496.0,54816509.0,NaN,NaN,54816510.0,54820397.0
160,CHR_HSCHR19KIR_FH06_BA1_HAP_CTG3_1,ENSG00000276423,KIR3DL1,protein_coding,1,54802859,54817155,ENST00000621353,KIR3DL1-228,protein_coding,True,tsl1,1857,54802859,54817155,True,NaN,P43629,P43629-1,Q5UCE2,1335.0,444.0,ENSP00000484972,54802859.0,54802877.0,54816653.0,54817155.0,54802878.0,54816652.0
202,CHR_HSCHR19KIR_FH15_A_HAP_CTG3_1,ENSG00000274948,KIR3DL1,protein_coding,1,54816448,54830767,ENST00000613229,KIR3DL1-213,protein_coding,True,tsl1,1860,54816448,54830767,True,NaN,NaN,NaN,Q5UCD9,1335.0,444.0,ENSP00000482160,54816448.0,54816510.0,54830306.0,54830767.0,54816511.0,54830305.0
207,CHR_HSCHR19KIR_RSH_BA2_HAP_CTG3_1,ENSG00000276501,KIR3DL1,protein_coding,1,54861074,54875379,ENST00000619476,KIR3DL1-229,protein_coding,True,tsl1,1865,54861074,54875379,True,NaN,NaN,NaN,Q5UCD8,1335.0,444.0,ENSP00000478871,54861074.0,54861100.0,54874877.0,54875379.0,54861101.0,54874876.0
222,CHR_HSCHR19KIR_ABC08_AB_HAP_T_P_CTG3_1,ENSG00000275288,KIR3DL1,protein_coding,1,54816438,54830737,ENST00000616188,KIR3DL1-214,protein_coding,True,tsl1,1860,54816438,54830737,True,NaN,P43629,P43629-1,Q5UCE2,1335.0,444.0,ENSP00000484036,54816438.0,54816500.0,54830276.0,54830737.0,54816501.0,54830275.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40031,CHR_HSCHR19KIR_7191059-2_CTG3_1,ENSG00000284093,KIR3DL1,protein_coding,1,54816446,54830759,ENST00000640606,KIR3DL1-256,protein_coding,False,tslNA,1800,54816465,54830758,True,NaN,NaN,NaN,Q15702,1284.0,427.0,ENSP00000492170,54816465.0,54816478.0,54830257.0,54830758.0,54816479.0,54830256.0
40036,CHR_HSCHR19KIR_7191059-2_CTG3_1,ENSG00000284093,KIR3DL1,protein_coding,1,54816446,54830759,ENST00000640581,KIR3DL1-255,protein_coding,False,tslNA,1564,54816468,54830759,True,NaN,NaN,NaN,A0A1W2PRK8,1050.0,349.0,ENSP00000492469,54816468.0,54816478.0,54830257.0,54830759.0,54816479.0,54830256.0
232711,19,ENSG00000167633,KIR3DL1,protein_coding,1,54816468,54830778,ENST00000391728,KIR3DL1-203,protein_coding,True,tsl1,1871,54816468,54830778,True,principal2,P43629,P43629-1,Q5UCE2,1335.0,444.0,ENSP00000375608,54816468.0,54816500.0,54830276.0,54830778.0,54816501.0,54830275.0
232713,19,ENSG00000167633,KIR3DL1,protein_coding,1,54816468,54830778,ENST00000326542,KIR3DL1-201,protein_coding,False,tsl1,1800,54816487,54830777,True,alternative2,NaN,NaN,W5QJC1,1284.0,427.0,ENSP00000326868,54816487.0,54816500.0,54830276.0,54830777.0,54816501.0,54830275.0


In [321]:
cds_pos_arr[[0,1], :].sum(axis=0)[mask] - seq_pos_arr[mask]

array([0, 0, 0])

In [311]:
c = Counter(cds_pos_arr[(0,1), :].sum(axis=0))

In [312]:
c

Counter({1: 2, 0: 2, 2: 1})

In [295]:
np.where(np.array([0,1,0,0,1,1]) == 1)[0]

array([1, 4, 5], dtype=int64)

In [304]:
np.array([1,0,0]) - np.array([1,0,1])

array([ 0,  0, -1])

In [267]:

np.where(cds_pos_arr2 == 1, cds_pos_arr2.sum(axis=0), 0)

array([[1, 0, 0, 0, 2],
       [0, 1, 0, 0, 2],
       [0, 0, 1, 1, 0]])

In [268]:
seq_pos_arr2 = seq_pos_arr[:, cds_mask[0]].sum(axis=0)

In [269]:
seq_pos_arr2

array([1, 1, 1, 3, 2])

In [258]:
np.subtract(cds_pos_arr[:, cds_mask[0]], seq_pos_arr[:, cds_mask[0]])

array([[ 0,  0,  0, -1,  0],
       [ 0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0]])

In [208]:
cds_pos_arr[:, cds_pos_arr.any(0)]

array([[1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0]])

In [193]:
(gene_view.iloc[i, [-2,-1]].astype(int) - gene_start).to_list()

[15602, 26941]

In [195]:
cds_pos_arr[0, [1,2]] = 1

In [222]:
np.r_[eval('0:266, 288:300')]

SyntaxError: invalid syntax (<string>, line 1)

In [232]:
np.r_[slice(0, 267, None), slice(20018, 20336, None)]

array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
           9,    10,    11,    12,    13,    14,    15,    16,    17,
          18,    19,    20,    21,    22,    23,    24,    25,    26,
          27,    28,    29,    30,    31,    32,    33,    34,    35,
          36,    37,    38,    39,    40,    41,    42,    43,    44,
          45,    46,    47,    48,    49,    50,    51,    52,    53,
          54,    55,    56,    57,    58,    59,    60,    61,    62,
          63,    64,    65,    66,    67,    68,    69,    70,    71,
          72,    73,    74,    75,    76,    77,    78,    79,    80,
          81,    82,    83,    84,    85,    86,    87,    88,    89,
          90,    91,    92,    93,    94,    95,    96,    97,    98,
          99,   100,   101,   102,   103,   104,   105,   106,   107,
         108,   109,   110,   111,   112,   113,   114,   115,   116,
         117,   118,   119,   120,   121,   122,   123,   124,   125,
         126,   127,